In [22]:
import os, torch, librosa, sklearn, librosa.display
import numpy as np, pandas as pd, matplotlib.pyplot as plt
from sklearn import preprocessing
from IPython.display import Audio
from tqdm.auto import tqdm

random_state = np.random.RandomState(0)

data_path = '/kaggle/input/gtzan-dataset-music-genre-classification/Data'
print(list(os.listdir(f'{data_path}/genres_original/')))

['disco', 'metal', 'reggae', 'blues', 'rock', 'classical', 'jazz', 'hiphop', 'country', 'pop']


In [23]:
data = pd.read_csv(f'{data_path}/features_30_sec.csv')
data = data.iloc[0:, 1:] 
data.head()

y = data['label'] # genre variable.
X = data.loc[:, data.columns != 'label'] #select all columns but not the labels

#### NORMALIZE X ####

# Normalize so everything is on the same scale. 

cols = X.columns
min_max_scaler = preprocessing.MinMaxScaler()
np_scaled = min_max_scaler.fit_transform(X)

# new data frame with the new scaled data. 
X = pd.DataFrame(np_scaled, columns = cols)

In [24]:
from sklearn.model_selection import train_test_split
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import classification_report, accuracy_score

# Step 1: Prepare the data
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=random_state)

# Step 2: Select the model
knn = KNeighborsClassifier(n_neighbors=4) 

# Step 4: Train the model
knn.fit(X_train, y_train)

# Step 5: Evaluate the model
y_pred = knn.predict(X_test)
print("Accuracy:", accuracy_score(y_test, y_pred))
print("Classification Report:")
print(classification_report(y_test, y_pred))


Accuracy: 0.69
Classification Report:
              precision    recall  f1-score   support

       blues       0.75      0.78      0.76        27
   classical       0.66      1.00      0.79        21
     country       0.58      0.64      0.61        33
       disco       0.43      0.62      0.51        29
      hiphop       0.77      0.68      0.72        34
        jazz       0.89      0.61      0.72        28
       metal       0.92      0.74      0.82        31
         pop       0.92      0.71      0.80        34
      reggae       0.60      0.58      0.59        26
        rock       0.65      0.65      0.65        37

    accuracy                           0.69       300
   macro avg       0.72      0.70      0.70       300
weighted avg       0.72      0.69      0.70       300

